# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118126e+02     1.716112e+00
 * time: 0.08417487144470215
     1     1.050606e+01     8.847660e-01
 * time: 1.525364875793457
     2    -1.218308e+01     1.027633e+00
 * time: 1.61049485206604
     3    -3.402191e+01     9.033674e-01
 * time: 1.723024845123291
     4    -4.754282e+01     6.674977e-01
 * time: 1.83748197555542
     5    -5.684587e+01     2.415073e-01
 * time: 1.9407768249511719
     6    -5.977847e+01     1.578673e-01
 * time: 2.0326569080352783
     7    -6.090650e+01     5.394732e-02
 * time: 2.109747886657715
     8    -6.133707e+01     8.134625e-02
 * time: 2.1900758743286133
     9    -6.160888e+01     4.436375e-02
 * time: 2.2664198875427246
    10    -6.182841e+01     2.756352e-02
 * time: 2.3499269485473633
    11    -6.198800e+01     2.178306e-02
 * time: 2.427133798599243
    12    -6.205186e+01     1.567220e-02
 * time: 2.502992868423462
    13    -6.211382e+01     1.367945e-02
 * time: 2.585162878036499
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671079
    AtomicLocal         -18.8557738
    AtomicNonlocal      14.8522682
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485391 
    Xc                  -19.3336828

    total               -62.261666461029
